Copy all the necessary Libraries

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from skimage.feature import hog
get_ipython().magic('matplotlib inline')
from scipy.ndimage.measurements import label
import csv

Preprocess the data used for training and testing various machine learning models
Data downloaded from University of California website https://archive.ics.uci.edu/ml/datasets/Poker+Hand

In [2]:
with open('./poker_train_data1.csv', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    #next(reader, None)  # skip the header
    xxx=[]
    for line in reader:
        xxx.append(line)
with open('./poker_out3.csv', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    #next(reader, None)  # skip the header
    yyy=[]
    for line in reader:
        yyy.append(line)
#Augmenting the data by repeting the same data, so that every class should have minumum 2000 examples
repeat = [1,1,2,5,21,41,41,401,401,401]
xx = []
yy = []
for x,y in zip(xxx,yyy):
    for i in range(repeat[int(y[0])]):
        xx.append(x)
        yy.append(y)
print(len(xx))
print(len(yy))
#Connvert the data into numpy array
X= np.asarray(xx)
Y= np.asarray(yy)
print(X.shape)
print(Y.shape)
print(X[0])
print(Y[0])
#Connvert the data to integer
X = X.astype(np.int32)
Y = Y.astype(np.int32)
YY = np.copy(Y)
#One hot encoding for the catogorical data
ohe = OneHotEncoder(sparse=False)
Y = ohe.fit_transform(Y)
X = ohe.fit_transform(X)
print(X.shape)
print(Y.shape)
print(YY.shape)

39727
39727
(39727, 10)
(39727, 1)
['2' '11' '2' '13' '2' '10' '2' '12' '2' '1']
['9']
(39727, 85)
(39727, 10)
(39727, 1)


Preprocess the data used fro validating the models

In [3]:
with open('./datpoker_X.csv', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    #next(reader, None)  # skip the header
    xxxx=[]
    for line in reader:
        xxxx.append(line)
with open('./datpoker_y.csv', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    #next(reader, None)  # skip the header
    yyyy=[]
    for line in reader:
        yyyy.append(line)
print(len(xxxx))
print(len(yyyy))
#Connvert the data into numpy array
X_val= np.asarray(xxxx)
Y_val= np.asarray(yyyy)
print(X_val.shape)
print(Y_val.shape)
#Connvert the data to integer
X_val = X_val.astype(np.int32)
Y_val = Y_val.astype(np.int32)
YY_val = np.copy(Y_val)
#One hot encoding for the catogorical data
ohe1 = OneHotEncoder(sparse=False)
Y_val = ohe1.fit_transform(Y_val)
X_val = ohe1.fit_transform(X_val)
print(X_val.shape)
print(Y_val.shape)

239029
239029
(239029, 10)
(239029, 1)
(239029, 85)
(239029, 10)


Split the data into Train and Test (ratio 80:20)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=22)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, YY, test_size=0.2, random_state=22)

SVC Linear Model - giving very poor accuracy

In [5]:
svc = LinearSVC(loss='hinge') # Use a linear SVC 
t=time.time() # Check the training time for the SVC
svc.fit(X_train1, y_train1) # Train the classifier
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
print('Valid Accuracy of SVC = ', svc.score(X_test1, y_test1))
print('Test Accuracy of SVC = ', svc.score(X_val, YY_val))

C:\Users\dsingh\AppData\Local\Continuum\Anaconda3\envs\carnd-term11\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.08 Seconds to train SVC...
Valid Accuracy of SVC =  0.216586961993
Test Accuracy of SVC =  0.0314104146359


Naive Bays model, very low accuracy

In [6]:
clf =  GaussianNB()
t=time.time() # Check the training time for the SVC
clf.fit(X_train1, y_train1)
t2 = time.time()
pred=clf.predict(X_test1)
pred1=clf.predict(X_val)
print(round(t2-t, 2), 'Seconds to train NB...')
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test1, pred)
acc1=accuracy_score(YY_val, pred1)
print("Valid Accuracy of NB : " , acc)
print("Test Accuracy of NB : " , acc1)

C:\Users\dsingh\AppData\Local\Continuum\Anaconda3\envs\carnd-term11\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.04 Seconds to train NB...
Valid Accuracy of NB :  0.231059652655
Test Accuracy of NB :  0.0214911161407


Decision tree model - moderate accuracy

In [7]:
from sklearn import tree
clf1 = tree.DecisionTreeClassifier()
clf1.fit(X_train1, y_train1)
pred2=clf1.predict(X_test1)
pred3=clf1.predict(X_val)
from sklearn.metrics import accuracy_score
acc2=accuracy_score(y_test1, pred2)
acc3=accuracy_score(YY_val, pred3)
print(acc2)
print(acc3)

0.751069720614
0.582707537579


Deep neural network with 3 layers - Giving 99.75% accuracy

In [8]:
from keras.models import Sequential
from keras.layers import Dense

# create model
model = Sequential()
model.add(Dense(30, input_dim=85, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, batch_size=30, nb_epoch=150)
# evaluate the model
scores = model.evaluate(X_test, y_test)
print("Test Accuracy")
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
scores1 = model.evaluate(X_val, Y_val)
print("Valid Accuracy")
print("\n%s: %.2f%%" % (model.metrics_names[1], scores1[1]*100))

Using TensorFlow backend.


Epoch 1/150
31781/31781 [==============================] - 1s - loss: 0.2202 - acc: 0.9117     
Epoch 2/150
31781/31781 [==============================] - 0s - loss: 0.1689 - acc: 0.9225     - ETA: 0s - loss
Epoch 3/150
31781/31781 [==============================] - 1s - loss: 0.1495 - acc: 0.9294     
Epoch 4/150
31781/31781 [==============================] - 1s - loss: 0.1322 - acc: 0.9371     -  - ETA: 0s  - ETA: 0s -
Epoch 5/150
31781/31781 [==============================] - 1s - loss: 0.1199 - acc: 0.9424     - ETA: - ETA: 0s - loss: 0.1204 - acc: - ETA: 0s - loss: 0.1
Epoch 6/150
31781/31781 [==============================] - 1s - loss: 0.1114 - acc: 0.9470     
Epoch 7/150
31781/31781 [==============================] - 1s - loss: 0.1051 - acc: 0.9511     
Epoch 8/150
31781/31781 [==============================] - 1s - loss: 0.1002 - acc: 0.9540     - ETA: - ETA: 0s - loss: 0.1003 - acc: 0.9
Epoch 9/150
31781/31781 [==============================] - 1s - loss: 0.0955 - acc: 0.956

31781/31781 [==============================] - 1s - loss: 0.0036 - acc: 0.9990     
Epoch 74/150
31781/31781 [==============================] - 1s - loss: 0.0027 - acc: 0.9992     - ETA: 0s - loss: 0.0027 -
Epoch 75/150
31781/31781 [==============================] - 1s - loss: 0.0033 - acc: 0.9989     - ETA: 0s - loss: 0. - ETA: 0s - loss: 0.0033 - acc: - ETA: 0s 
Epoch 76/150
31781/31781 [==============================] - 1s - loss: 0.0029 - acc: 0.9992     - ETA: 0s - loss: 0.0029 - a
Epoch 77/150
31781/31781 [==============================] - 1s - loss: 0.0024 - acc: 0.9993     - ETA: 0s - loss: 0 - ETA: 0s - loss: 0.0024 - ac
Epoch 78/150
31781/31781 [==============================] - 1s - loss: 0.0032 - acc: 0.9990     
Epoch 79/150
31781/31781 [==============================] - 1s - loss: 0.0032 - acc: 0.9989     - ETA: 0s - lo - ETA: 0s - loss: 0.0026 - acc: 0. - ET
Epoch 80/150
31781/31781 [==============================] - 1s - loss: 0.0024 - acc: 0.9994     - ETA: 0s - loss: 

31781/31781 [==============================] - 1s - loss: 0.0012 - acc: 0.9996     - ETA: 0s - loss: 0.0011 -  - ETA: 0s - loss: 0.0012 - a
Epoch 145/150
31781/31781 [==============================] - 1s - loss: 0.0014 - acc: 0.9995         - ETA: 0s - loss: 4.7896e-04 - acc: 0.999 - ETA: 0s - loss: 5.0201e-04 - ac
Epoch 146/150
31781/31781 [==============================] - 1s - loss: 0.0015 - acc: 0.9995         - ETA: 0s - loss: 7.2575e-04 - 
Epoch 147/150
31781/31781 [==============================] - 1s - loss: 9.6126e-04 - acc: 0.9997     
Epoch 148/150
31781/31781 [==============================] - 1s - loss: 4.1860e-04 - acc: 0.9999     - ETA: 0s - loss: 4.1851e-04 - acc: 0.99
Epoch 149/150
31781/31781 [==============================] - 1s - loss: 0.0022 - acc: 0.9992     
Epoch 150/150
7456/7946 [===========================>..] - ETA: 0sTest Accuracy

acc: 99.75%
238464/239029 [============================>.] - ETA: 0sValid Accuracy

acc: 99.42%


Saving the model

In [9]:
model.save_weights('poker_model_weights.h5')
model.save('my_model.h5')

In [10]:
weights = []
for layer in model.layers:
    weights.append(layer.get_weights()) # list of numpy arrays

In [11]:
wts = np.asarray(weights)
print(wts[0][1].shape)

(30,)
